## Comparing simulation of IDM with simulation of ACC

In [ ]:
# Do the necessary imports
import matplotlib.pyplot as plt
import numpy as np
from simulation import SimulationLeadBraking, IDMPlus, idm_lead_braking_pars, ACC, \
    acc_lead_braking_pars, SimulationApproaching, idm_approaching_pars, acc_approaching_pars, \
    SimulationCutIn, idm_cutin_pars, acc_cutin_pars

## Functions for comparison

In [ ]:
def get_features_sim_data(data):
    """ Get all kinds of features from the simulation data. 
    
    The following information is extracted.
    - Root mean square of the acceleration
    - Maximum acceleration
    - Minimum acceleration
    - Mean speed
    - Minimum gap
    - Minimum THW
    - Minimum TTC (maximum set at 10 seconds)
    
    :param data: The data obtained from StringSimulator.simulation.
    :return: Vector with information as described above.
    """
    ego_acceleration = data[:, 1, 2]
    ego_speed = data[:, 1, 1]
    lead_speed = data[:, 0, 1]
    gap = data[:, 0, 0] - data[:, 1, 0]
    gap[gap < 0] = 0
    indices_ttc = ego_speed > lead_speed
    features = [
        np.sqrt(np.mean(ego_acceleration**2)),  # RMS acceleration
        np.max(ego_acceleration),  # Maximum acceleration
        np.min(ego_acceleration),  # Minimum acceleration
        np.mean(ego_speed),  # Mean speed
        np.min(gap),  # Minimum gap
        np.min(gap[ego_speed > 0] / ego_speed[ego_speed > 0]),  # Minimum THW
        10 if not np.any(indices_ttc) else \
            min(np.min(gap[indices_ttc] / (ego_speed[indices_ttc] - lead_speed[indices_ttc])), 
                10)  # Minimum TTC
    ]
    return np.array(features)

In [ ]:
WEIGHTS = np.array([10, 5, 5, 1, .5, 5, 1])
def difference(data1, data2):
    return np.sqrt(np.sum((WEIGHTS * (get_features_sim_data(data1) - 
                                      get_features_sim_data(data2)))**2))

## Example leading vehicle decelerating

In [ ]:
simulator_idm = SimulationLeadBraking(follower=IDMPlus(), follower_parameters=idm_lead_braking_pars,
                                      min_simulation_time=5)
simulator_acc = SimulationLeadBraking(follower=ACC(), follower_parameters=acc_lead_braking_pars,
                                      min_simulation_time=5, comparison_simulator=simulator_idm)

In [ ]:
K1_ACCS = np.linspace(0.1, 1, 100)
# Note that a small reactiontime of the "human" is used, because it is assumed that the "ideal"
# ACC acts like a human with a superb reaction time.
SCENARIO_PARS = dict(v0=20, amean=3, dv=10, reactiontime=0.1)
def test_k1():
    differences = np.zeros_like(K1_ACCS)
    for i, k1_acc in enumerate(K1_ACCS):
        SCENARIO_PARS.update(dict(k1_acc=k1_acc))
        data1, data2 = simulator_acc.comparison(SCENARIO_PARS)
        differences[i] = difference(data1, data2)
    
    plt.plot(K1_ACCS, differences)
    plt.xlabel("Distance gain ACC")
    plt.ylabel("Difference with IDM")
    _ = plt.title("Optimum is {:.1f} at a gain of {:.2f}".format(np.min(differences),
                                                                 K1_ACCS[np.argmin(differences)]))
test_k1()

In [ ]:
K2_ACCS = np.linspace(0.02, 0.5, 100)
SCENARIO_PARS = dict(v0=20, amean=3, dv=10, reactiontime=0.1)
def test_k2():
    differences = np.zeros_like(K2_ACCS)
    for i, k2_acc in enumerate(K2_ACCS):
        SCENARIO_PARS.update(dict(k2_acc=k2_acc))
        data1, data2 = simulator_acc.comparison(SCENARIO_PARS)
        differences[i] = difference(data1, data2)
    
    plt.plot(K2_ACCS, differences)
    plt.xlabel("Speed gain ACC")
    plt.ylabel("Difference with IDM")
    _ = plt.title("Optimum is {:.1f} at a gain of {:.2f}".format(np.min(differences),
                                                                 K2_ACCS[np.argmin(differences)]))

test_k2()
# Note that result will most likely be difference if sensor noise is considered.

## Example approaching slower vehicle

In [ ]:
simulator_idm = SimulationApproaching(followers=[IDMPlus()], 
                                      followers_parameters=[idm_approaching_pars],
                                      min_simulation_time=5)
simulator_acc = SimulationApproaching(followers=[ACC()], followers_parameters=[acc_approaching_pars],
                                      min_simulation_time=5, comparison_simulator=simulator_idm)

In [ ]:
SCENARIO_PARS = dict(vego=15, ratio_vtar_vego=0.4, reactiontime=0.1)
test_k1()
# Not sure what the peak is supposed to mean...

In [ ]:
SCENARIO_PARS = dict(vego=15, ratio_vtar_vego=0.4, reactiontime=0.1)
test_k2()

## Example cut-in

In [ ]:
simulator_idm = SimulationCutIn(follower=IDMPlus(), follower_parameters=idm_cutin_pars,
                                      min_simulation_time=5)
simulator_acc = SimulationCutIn(follower=ACC(), follower_parameters=acc_cutin_pars,
                                min_simulation_time=5, comparison_simulator=simulator_idm)

In [ ]:
SCENARIO_PARS = dict(dinit=20, vlead=10, vego=15, reactiontime=0.1)
test_k1()

In [ ]:
SCENARIO_PARS = dict(dinit=20, vlead=10, vego=15, reactiontime=0.1)
test_k2()